In [ ]:
!pip uninstall -y transformers tokenizers huggingface-hub
!pip install -q diffusers==0.30.0 transformers==4.30.2 accelerate safetensors opencv-python pillow gradio==4.44.0

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  error: subprocess-exited-with-error
  
  × Building wheel for tokenizers (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for tokenizers
ERROR: ERROR: Failed to build installable wheels for some pyproject.toml based projects (tokenizers)


In [ ]:
import torch
from diffusers import StableDiffusionPipeline, StableDiffusionControlNetPipeline, ControlNetModel
from diffusers import UniPCMultistepScheduler
from PIL import Image
import cv2
import numpy as np
import gradio as gr
import random
import os

device = "cuda" if torch.cuda.is_available() else "cpu"
device

RuntimeError: Failed to import diffusers.pipelines.stable_diffusion.pipeline_stable_diffusion because of the following error (look up to see its traceback):
cannot import name 'FLAX_WEIGHTS_NAME' from 'transformers.utils' (/usr/local/lib/python3.12/dist-packages/transformers/utils/__init__.py)

In [ ]:
# Base text-to-image model
base_model_id = "runwayml/stable-diffusion-v1-5"

# ControlNet model for edge/line guidance
controlnet_model_id = "lllyasviel/sd-controlnet-canny"

torch_dtype = torch.float16 if device == "cuda" else torch.float32

base_pipe = StableDiffusionPipeline.from_pretrained(
    base_model_id,
    torch_dtype=torch_dtype,
    safety_checker=None
).to(device)

controlnet = ControlNetModel.from_pretrained(
    controlnet_model_id,
    torch_dtype=torch_dtype
)

controlnet_pipe = StableDiffusionControlNetPipeline.from_pretrained(
    base_model_id,
    controlnet=controlnet,
    torch_dtype=torch_dtype,
    safety_checker=None
).to(device)

# Use a fast scheduler
base_pipe.scheduler = UniPCMultistepScheduler.from_config(base_pipe.scheduler.config)
controlnet_pipe.scheduler = UniPCMultistepScheduler.from_config(controlnet_pipe.scheduler.config)

In [ ]:
def resize_to_max(image: Image.Image, max_side: int = 768) -> Image.Image:
    w, h = image.size
    scale = max_side / max(w, h)
    if scale >= 1:
        return image
    new_size = (int(w * scale), int(h * scale))
    return image.resize(new_size, Image.LANCZOS)


def pil_to_cv2(image: Image.Image):
    return cv2.cvtColor(np.array(image), cv2.COLOR_RGB2BGR)


def cv2_to_pil(image):
    return Image.fromarray(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))


def canny_edges(image: Image.Image, low: int = 80, high: int = 160) -> Image.Image:
    img = pil_to_cv2(image)
    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    edges = cv2.Canny(img_gray, low, high)
    edges_rgb = cv2.cvtColor(edges, cv2.COLOR_GRAY2RGB)
    return cv2_to_pil(edges_rgb)


def to_clean_line_drawing(image: Image.Image, threshold: int = 220) -> Image.Image:
    """
    Simple post-process to push result toward a black‑on‑white line drawing.
    """
    img = pil_to_cv2(image)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    _, bw = cv2.threshold(gray, threshold, 255, cv2.THRESH_BINARY)
    bw_rgb = cv2.cvtColor(bw, cv2.COLOR_GRAY2RGB)
    return cv2_to_pil(bw_rgb)

In [ ]:
def view_prompt_prefix(view_type: str) -> str:
    view_type = (view_type or "").lower()
    if "plan" in view_type:
        return "architectural floor plan, top view, clean black line drawing, no shading, white background, "
    if "section" in view_type:
        return "architectural section cut, cut line, interior spaces visible, clean black line drawing, "
    if "elevation" in view_type:
        return "architectural elevation, orthographic front view, clean black line drawing, "
    if "axon" in view_type or "axonometric" in view_type or "isometric" in view_type:
        return "axonometric architectural drawing, clean black linework, "
    return "architectural line drawing, clean technical drafting style, "


def build_prompt(text_prompt: str, view_type: str) -> str:
    base = view_prompt_prefix(view_type)
    return base + (text_prompt or "").strip()

In [ ]:
def generate_drawing(
    text_prompt: str,
    input_image: Image.Image,
    mode: str,
    view_type: str,
    steps: int = 25,
    guidance_scale: float = 7.5,
    strength: float = 0.8,
    seed: int = -1,
):
    """
    mode:
      - 'Text only'
      - 'Sketch / rough image'
      - 'Reference image (edges only)'
    """
    if not text_prompt:
        return None

    prompt = build_prompt(text_prompt, view_type)

    if seed is None or seed < 0:
        seed = random.randint(0, 10_000)
    generator = torch.Generator(device=device).manual_seed(seed)

    # TEXT ONLY
    if mode == "Text only" or input_image is None:
        result = base_pipe(
            prompt=prompt,
            num_inference_steps=steps,
            guidance_scale=guidance_scale,
            generator=generator
        ).images[0]
        return to_clean_line_drawing(result)

    # IMAGE‑GUIDED MODES
    img = resize_to_max(input_image.convert("RGB"), 768)

    if mode == "Reference image (edges only)" or mode == "Sketch / rough image":
        # Use Canny edges as ControlNet condition
        edge_img = canny_edges(img)

        result = controlnet_pipe(
            prompt=prompt,
            image=edge_img,
            num_inference_steps=steps,
            guidance_scale=guidance_scale,
            generator=generator
        ).images[0]

        return to_clean_line_drawing(result)

    # Fallback
    result = base_pipe(
        prompt=prompt,
        image=img,
        strength=strength,
        num_inference_steps=steps,
        guidance_scale=guidance_scale,
        generator=generator
    ).images[0]

    return to_clean_line_drawing(result)

In [ ]:
def ui_generate(text_prompt, input_image, mode, view_type, steps, guidance, strength, seed):
    return generate_drawing(
        text_prompt=text_prompt,
        input_image=input_image,
        mode=mode,
        view_type=view_type,
        steps=steps,
        guidance_scale=guidance,
        strength=strength,
        seed=seed,
    )


with gr.Blocks(title="AI Drafting Assistant") as demo:
    gr.Markdown(
        """
        # AI Drafting Assistant
        Generate architectural line drawings (plans, sections, elevations) from text prompts and rough images.
        """
    )

    with gr.Row():
        with gr.Column():
            text_prompt = gr.Textbox(
                label="Design brief / description",
                placeholder="e.g. Small two-storey studio with double-height living space and mezzanine...",
                lines=4,
            )

            mode = gr.Radio(
                choices=["Text only", "Sketch / rough image", "Reference image (edges only)"],
                value="Text only",
                label="Input mode",
            )

            view_type = gr.Dropdown(
                choices=[
                    "Generic line drawing",
                    "Plan",
                    "Section",
                    "Elevation",
                    "Axonometric"
                ],
                value="Generic line drawing",
                label="View type",
            )

            input_image = gr.Image(
                label="Optional sketch / reference image",
                type="pil"
            )

            steps = gr.Slider(10, 40, value=25, step=1, label="Inference steps")
            guidance = gr.Slider(3.0, 12.0, value=7.5, step=0.5, label="Guidance scale")
            strength = gr.Slider(0.3, 1.0, value=0.8, step=0.05, label="Image influence (for img‑guided)")
            seed = gr.Number(value=-1, precision=0, label="Seed (-1 for random)")

            run_btn = gr.Button("Generate line drawing")

        with gr.Column():
            output_image = gr.Image(label="Output line drawing", type="pil")

    run_btn.click(
        fn=ui_generate,
        inputs=[text_prompt, input_image, mode, view_type, steps, guidance, strength, seed],
        outputs=[output_image],
    )

demo.launch(debug=False)